## Build Time - Tool Validation Example

Demonstrates using Tool Validation from ALTK to validate Python tools by running test cases with a ReAct agent bound to the required tools. After execution, the tool logs are analyzed to identify error types and provide corresponding recommendations.

#### Python Tool:

In [1]:
import os

os.environ["WX_API_KEY"] = "xxxx"
os.environ["WX_PROJECT_ID"] = "xxxx"

In [2]:
python_tool = '''
from enum import Enum
from typing import ClassVar, Optional, Type
import datetime
from dataclasses import dataclass
from langchain_core.tools import tool

class TimeOffTypes(Enum):
    """Represents the time off event types"""

    ABSENCE = "ABSENCE"
    PUBLIC_HOLIDAY = "PUBLIC_HOLIDAY"
    NON_WORKING_DAY = "NON_WORKING_DAY"

@dataclass
class UpcomingTimeOff:
    """Represents an upcoming time off event"""

    title: str
    start_date: str
    end_date: str
    start_time: Optional[str]
    end_time: Optional[str]
    duration: int
    time_unit: str
    cross_midnight: bool
    type: str
    status_formatted: Optional[str]
    absence_duration_category: Optional[str]

    #Schema: ClassVar[Type[Schema]] = Schema

@dataclass
class UpcomingTimeOffResponse:
    """Represents the response from getting a user's upcoming time off."""

    time_off_events: list[UpcomingTimeOff]

    #Schema: ClassVar[Type[Schema]] = Schema


@tool
def get_upcoming_time_off(
    user_id: str, start_date: str, end_date: str, time_off_types: list[str]
) -> UpcomingTimeOffResponse:
    """
    Retrieves the user's upcoming time off details from SAP SuccessFactors.

    """

    # Check for date format
    try:
        datetime.datetime.strptime(start_date, "%Y-%m-%d")
        datetime.datetime.strptime(end_date, "%Y-%m-%d")
    except ValueError:
        return {"error": "Invalid date format. Please use YYYY-MM-DD."}

    time_off_events=[]

    for  time_type in time_off_types:
    # Hard-coded values for testing
        if time_type == "ABSENCE":
            time_off_events.append(
                UpcomingTimeOff(
                    title="Vacation",
                    start_date="2024-01-01",
                    end_date="2024-01-05",
                    start_time=None,
                    end_time=None,
                    duration=5,
                    time_unit="DAYS",
                    cross_midnight=False,
                    type="ABSENCE",
                    status_formatted=None,
                    absence_duration_category=None,
                ) )
        elif time_type == "PUBLIC_HOLIDAY":
            time_off_events.append(
                UpcomingTimeOff(
                    title="New Year's Day",
                    start_date="2024-01-01",
                    end_date="2024-01-01",
                    start_time=None,
                    end_time=None,
                    duration=1,
                    time_unit="DAYS",
                    cross_midnight=False,
                    type="PUBLIC_HOLIDAY",
                    status_formatted=None,
                    absence_duration_category=None,
                ))
            time_off_events.append (UpcomingTimeOff(
                    title="Christmas Day",
                    start_date="2024-12-25",
                    end_date="2024-12-25",
                    start_time=None,
                    end_time=None,
                    duration=1,
                    time_unit="DAYS",
                    cross_midnight=False,
                    type="PUBLIC_HOLIDAY",
                    status_formatted=None,
                    absence_duration_category=None,
                ))

        elif time_type == "NON_WORKING_DAY":
            time_off_events.append(
                UpcomingTimeOff(
                    title="Weekend",
                    start_date="2024-01-06",
                    end_date="2024-01-07",
                    start_time=None,
                    end_time=None,
                    duration=2,
                    time_unit="DAYS",
                    cross_midnight=False,
                    type="NON_WORKING_DAY",
                    status_formatted=None,
                    absence_duration_category=None,
                ))

    
    if (len(time_off_events)==0):
        return {"status_code": 400, "error": "Invalid values for time off types"}
    else:
        return UpcomingTimeOffResponse(time_off_events)
'''
python_tool_name = "get_upcoming_time_off"

# can be obtained using Tool Test Case Generation Component
tool_test_cases = [
    {
        "id": "TC_1",
        "input": "user_id((str)) := usr_5f7b3a2c1d4e \n"
        "start_date((str)) := 2023-01-01 \n"
        "end_date((str)) := 2023-01-10 \n"
        "time_off_types((list[str])) := ['vacation', "
        "'sick_leave'] \n",
        "input_parameters": {
            "end_date": ["2023-01-10"],
            "start_date": ["2023-01-01"],
            "time_off_types": [["vacation", "sick_leave"]],
            "user_id": ["usr_5f7b3a2c1d4e"],
        },
        "mandatory_params": ["user_id", "start_date", "time_off_types", "end_date"],
        "nl_utterance": [
            "Can you fetch the upcoming time off for "
            'user "usr_5f7b3a2c1d4e" between January '
            "1, 2023, and January 10, 2023, "
            "including vacation and sick leave "
            "details from SAP SuccessFactors?",
            "From SAP SuccessFactors, could you "
            "retrieve the time off information for "
            'user "usr_5f7b3a2c1d4e" for the period '
            "from 2023-01-01 to 2023-01-10, "
            "specifically for vacation and sick "
            "leave?",
            "I need the upcoming time off details "
            'for user "usr_5f7b3a2c1d4e" from '
            "January 1, 2023, to January 10, 2023, "
            "focusing on vacation and sick leave, "
            "all retrieved from SAP SuccessFactors.",
            "Please get the time off records for "
            'user "usr_5f7b3a2c1d4e" from 2023-01-01 '
            "to 2023-01-10, covering vacation and "
            "sick leave, using the SAP "
            "SuccessFactors system.",
            "Using SAP SuccessFactors, can you pull "
            "up the time off data for user "
            '"usr_5f7b3a2c1d4e" from January 1, '
            "2023, to January 10, 2023, specifically "
            "for vacation and sick leave?",
        ],
        "scenario_type": "positive",
    },
    {
        "id": "TC_3",
        "input": "start_date((str)) := 2023-01-01 \n"
        "end_date((str)) := 2023-01-10 \n"
        "time_off_types((list[str])) := ['vacation', "
        "'sick_leave'] \n",
        "input_parameters": {
            "end_date": ["2023-01-10"],
            "start_date": ["2023-01-01"],
            "time_off_types": [["vacation", "sick_leave"]],
        },
        "mandatory_params": ["start_date", "time_off_types", "end_date"],
        "nl_utterance": [
            "Fetch my upcoming time off from SAP "
            "SuccessFactors for vacation and sick "
            "leave between January 1, 2023, and "
            "January 10, 2023.",
            "Can you show me my scheduled time off "
            "for vacation and sick leave from "
            "January 1, 2023, to January 10, 2023, "
            "in SAP SuccessFactors?",
            "I need to see my upcoming leave details "
            "in SAP SuccessFactors for vacation and "
            "sick leave from January 1, 2023, "
            "through January 10, 2023.",
            "Get my upcoming time off records from "
            "SAP SuccessFactors for vacation and "
            "sick leave starting January 1, 2023, "
            "and ending January 10, 2023.",
            "Pull up my future time off in SAP "
            "SuccessFactors, specifically for "
            "vacation and sick leave, from January "
            "1, 2023, to January 10, 2023.",
        ],
        "scenario_type": "negative",
    },
]

## Define custom configurations

Creating a langgraph react agent with tools using WATSONX LLM Provider

In [3]:
# creating react agent with python tool
import importlib.util
import os
from ibm_watsonx_ai import Credentials as wx_credentials
from langchain_ibm import ChatWatsonx
from langgraph.prebuilt import create_react_agent


def get_python_tool(python_tool_string, python_tool_name):
    spec = importlib.util.spec_from_loader("tool_py", loader=None)
    tool_py = importlib.util.module_from_spec(spec)
    exec(python_tool_string, tool_py.__dict__)
    return tool_py.__getattribute__(python_tool_name)


# adding react agent llm code for WATSONX


def get_agent_llm(agent_llm_model_id="mistralai/mistral-medium-2505"):
    WATSONX_URL = os.getenv("WX_URL", "https://us-south.ml.cloud.ibm.com")
    WATSONX_API_KEY = os.getenv("WX_API_KEY", "")
    WATSONX_PROJECT = os.getenv("WX_PROJECT_ID", "")

    # set "WATSONX_API_KEY" env variable as required by ChatWatsonx Model
    os.environ["WATSONX_API_KEY"] = WATSONX_API_KEY
    credentials = wx_credentials(url=WATSONX_URL, api_key=WATSONX_API_KEY)
    project_id = WATSONX_PROJECT
    try:
        llm_parameters = {
            "decoding_method": "greedy",
            "max_new_tokens": 800,
            "min_new_tokens": 1,
        }
        wx_chat_llm = ChatWatsonx(
            model_id=agent_llm_model_id,
            url=WATSONX_URL,
            project_id=project_id,
            credentials=credentials,
            params=llm_parameters,
        )
        return wx_chat_llm
    except Exception as e:
        print(
            "Please check if all WatsonX related environment varaibles - WX_URL , WX_API_KEY , WX_PROJECT_ID are set"
        )
        print("Error in react agent llm configuration - ", e)


def get_react_agent(
    python_tool_string,
    python_tool_name,
    agent_llm_model_id="mistralai/mistral-medium-2505",
):
    tool = get_python_tool(python_tool_string, python_tool_name)
    tools = [tool]
    agent_llm = get_agent_llm(agent_llm_model_id)
    react_agent_with_tools = create_react_agent(agent_llm, tools)
    print("Created react agent with tools")
    return react_agent_with_tools


agent_with_tools = get_react_agent(
    python_tool,
    python_tool_name,
    agent_llm_model_id="meta-llama/llama-3-3-70b-instruct",
)

Created react agent with tools


/var/folders/rn/9r22xcbn7j5cmjdlz05f1l7r0000gn/T/ipykernel_49453/746484189.py:48: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  react_agent_with_tools = create_react_agent(agent_llm, tools)


### Tool Validation

In [4]:
from altk.core.toolkit import AgentPhase
from altk.build_time.tool_validation_toolkit.core.toolkit import ToolValidationInput
from altk.build_time.tool_validation_toolkit.core.config import ToolValidationConfig
from altk.build_time.tool_validation_toolkit.utils.tool_validation import (
    PythonToolValidationComponent,
)
import pprint

tool_validation_input = ToolValidationInput(
    python_tool_name=python_tool_name,
    tool_test_cases=tool_test_cases,
    agent_with_tools=agent_with_tools,
)
config = ToolValidationConfig(report_level="detailed")
tool_validation_middleware = PythonToolValidationComponent()
result = tool_validation_middleware.process(
    data=tool_validation_input, config=config, phase=AgentPhase.RUNTIME
)

pprint.pprint(result.test_report)

Exception occured in running agentic flow , Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://docs.langchain.com/oss/python/langgraph/errors/GRAPH_RECURSION_LIMIT
Exception occured in running agentic flow , Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://docs.langchain.com/oss/python/langgraph/errors/GRAPH_RECURSION_LIMIT


{'number_of_test_cases': 10,
 'tool_name': 'get_upcoming_time_off',
 'tool_test_cases': [{'test_error_recommendations': ['Tool input malformed, '
                                                     'please provide tool '
                                                     'inputs in correct '
                                                     'format'],
                      'test_error_taxonomy': ['Issue with the inputs provided '
                                              'to the tool'],
                      'test_status': 'Failed',
                      'test_utterance': 'Can you fetch the upcoming time off '
                                        'for user "usr_5f7b3a2c1d4e" between '
                                        'January 1, 2023, and January 10, '
                                        '2023, including vacation and sick '
                                        'leave details from SAP '
                                        'SuccessFactors?',
               

In [5]:
pprint.pprint(result.test_report["tool_test_cases"][5])

{'test_error_recommendations': ['Tool input malformed, please provide tool '
                                'inputs in correct format'],
 'test_error_taxonomy': ['Issue with the inputs provided to the tool'],
 'test_status': 'Failed',
 'test_utterance': 'Fetch my upcoming time off from SAP SuccessFactors for '
                   'vacation and sick leave between January 1, 2023, and '
                   'January 10, 2023.',
 'tool_execution_events': {'agentic_flow_events': [{'turn_event': '{"agent": '
                                                                  '{"messages": '
                                                                  '[{"lc": 1, '
                                                                  '"type": '
                                                                  '"constructor", '
                                                                  '"id": '
                                                                  '["langchain", '
            